# Step 1: Setup Kaggle Notebook & Keys

In [1]:

# Install required packages
!pip install -q -U google-generativeai langchain langchain-google-genai langchain-core chromadb sentence-transformers numpy pandas langchain-community

import os
import json
import re
import datetime
import time
import requests
import traceback
import numpy as np
import pandas as pd
from IPython.display import display, HTML, clear_output
from kaggle_secrets import UserSecretsClient
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import torch
from google.ai.generativelanguage import Part, FunctionResponse
from langchain.schema import Document

# --- Access Google API Key ---
user_secrets = UserSecretsClient()
try:
    GOOGLE_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")
    os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
    print("Google API Key retrieved successfully.")
except Exception as e:
    print(f"ERROR: Could not find the Kaggle Secret 'GOOGLE_API_KEY': {e}")
    raise ValueError("Missing Google API Key")

# --- Access Google Search Custom Search ID ---
GOOGLE_CSE_ID = None
try:
    # Make sure secret label matches EXACTLY what you have in Kaggle Secrets
    GOOGLE_CSE_ID = user_secrets.get_secret("GOOGLE_CSE_ID")
    os.environ['GOOGLE_CSE_ID'] = GOOGLE_CSE_ID
    print("Google Custom Search ID retrieved successfully.")
except Exception as e:
    print(f"ERROR: Could not find Google Custom Search ID: {e}")
    print("Some search-based features will be simulated.")

# Optional: Check if keys were loaded
if not GOOGLE_CSE_ID:
    print("WARNING: Google Custom Search ID is missing. Search function will use simulation.")

print("Setup Complete.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 69.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 60.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.6/340.6

# Step 2: Initialize the Gemini Client

Configure the Gemini client with your API key.

In [2]:
# === SECTION 2: INITIALIZE GEMINI CLIENT ===

# Configure the Gemini client with API key
if 'GOOGLE_API_KEY' in os.environ:
    genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
    
    # List available models to ensure we use one that exists
    print("Available models:")
    for m in genai.list_models():
        if 'generateContent' in m.supported_generation_methods:
            print(f"- {m.name}")
    
    # Choose a model - make sure we use a fully qualified name like "gemini-pro"
    # not something like "models/gemini-pro" which may cause format errors
    model = genai.GenerativeModel('gemini-2.5-pro-exp-03-25')
    print(f"Gemini Model '{model.model_name}' initialized.")
else:
    print("ERROR: Google API Key not set in environment variables.")

Available models:
- models/gemini-1.0-pro-vision-latest
- models/gemini-pro-vision
- models/gemini-1.5-pro-latest
- models/gemini-1.5-pro-001
- models/gemini-1.5-pro-002
- models/gemini-1.5-pro
- models/gemini-1.5-flash-latest
- models/gemini-1.5-flash-001
- models/gemini-1.5-flash-001-tuning
- models/gemini-1.5-flash
- models/gemini-1.5-flash-002
- models/gemini-1.5-flash-8b
- models/gemini-1.5-flash-8b-001
- models/gemini-1.5-flash-8b-latest
- models/gemini-1.5-flash-8b-exp-0827
- models/gemini-1.5-flash-8b-exp-0924
- models/gemini-2.5-pro-exp-03-25
- models/gemini-2.5-pro-preview-03-25
- models/gemini-2.0-flash-exp
- models/gemini-2.0-flash
- models/gemini-2.0-flash-001
- models/gemini-2.0-flash-exp-image-generation
- models/gemini-2.0-flash-lite-001
- models/gemini-2.0-flash-lite
- models/gemini-2.0-flash-lite-preview-02-05
- models/gemini-2.0-flash-lite-preview
- models/gemini-2.0-pro-exp
- models/gemini-2.0-pro-exp-02-05
- models/gemini-exp-1206
- models/gemini-2.0-flash-thinking

# Step 3: Define Core Data Structures & Context Cache

Itinerary Structure: Use a Python dictionary that can be easily converted to/from JSON.

Context Cache: A simple dictionary to store user preferences and conversation history.

In [3]:
# === SECTION 3: DEFINE CORE DATA STRUCTURES & CONTEXT CACHE ===

# Context Cache to store preferences, history, and current state
context_cache = {
    "user_preferences": {
        "legal_language_level": "Professional",  # Options: Professional, Plain English
        "citation_style": "Kenya Law Reports",
        "include_statutory_references": True,
        "detail_level": "Comprehensive",
        "jurisdiction": "Kenya"  # New field: Options: 'Kenya' or 'Germany'
    },
    "conversation_history": [],
    "current_case": None,  # Will hold the current case data
    "vector_stores": {  # Modified to hold multiple vector stores for different jurisdictions
        "kenya": None, 
        "germany": None
    },  
    "document_index": {  # Will now include jurisdiction in metadata
        "kenya": {},
        "germany": {}
    },  
    "search_results_cache": {}  # Cache for search results to avoid redundant API calls
}

# Example case structure for German cases
example_german_case_structure = {
    "case_name": "Schmidt v. Federal Republic of Germany",
    "citation": "BGH VIII ZR 42/20",
    "court": "Bundesgerichtshof (Federal Court of Justice)",
    "judge": "Senate panel led by Justice Weber",
    "date": "May 12, 2021",
    "parties": {
        "plaintiff": "Hans Schmidt",
        "defendant": "Federal Republic of Germany"
    },
    "summary": {
        "brief": "Challenge to rental deposit retention after lease termination.",
        "comprehensive": "...",  # Full comprehensive summary
        "legal_principles": ["Burden of proof in rental damage cases", "Proportionality principle"]
    },
    "analysis": {
        "constitutional_issues": ["Whether retention of deposit without proof violates property rights"],
        "holding": "Landlord bears burden of proof for claimed damages; mere allegation insufficient to retain deposit",
        "reasoning": "..."  # Detailed reasoning
    },
    "related_statutes": ["German Civil Code (BGB) §§ 535, 546, 551", "German Constitution (GG) Art. 14"],
    "related_cases": ["BGH VIII ZR 71/19"]
}

print("Data structures defined.")

Data structures defined.


# Step 4: Initial Itinerary Generation (Few-shot Prompting & Structured Output)

Create a function that takes user requirements.

Craft a prompt with few-shot examples showing the desired JSON structure and style (based on interests, budget, pace).

Instruct the model to output only valid JSON.

In [4]:
# === SECTION 4: CREATE SAMPLE DOCUMENTS AND VECTOR STORE ===

def create_sample_kenya_law_documents(output_dir="data/raw"):
    """
    Create sample Kenyan legal case files for demonstration purposes.
    """
    print(f"Creating sample Kenyan documents in {output_dir}...")
    os.makedirs(output_dir, exist_ok=True)
    
    # Sample cases
    sample_cases = [
        {
            "title": "Smith v. Republic of Kenya (2022)",
            "content": """
            REPUBLIC OF KENYA
            IN THE HIGH COURT OF KENYA AT NAIROBI
            CONSTITUTIONAL AND HUMAN RIGHTS DIVISION
            PETITION NO. 123 OF 2022
            
            BETWEEN
            
            JOHN SMITH....................................................................PETITIONER
            
            AND
            
            THE REPUBLIC OF KENYA.....................................................RESPONDENT
            
            JUDGMENT
            
            Introduction:
            1. The Petitioner, John Smith, filed this petition on 15th March 2022 challenging the constitutionality of Section 24 of the Public Order Act.
            
            Background:
            2. The Petitioner was arrested during a peaceful demonstration at Uhuru Park, Nairobi, on 10th February 2022.
            3. The Petitioner claims that the arrest violated his constitutional rights to freedom of assembly and expression.
            
            Issues for Determination:
            4. Whether Section 24 of the Public Order Act contravenes Articles 33 and 37 of the Constitution.
            
            Analysis:
            5. The Court has considered the submissions made by both parties. The Constitution guarantees every person the right to freedom of expression and assembly.
            6. The limitation imposed by Section 24 of the Public Order Act must be justified in an open and democratic society.
            
            Conclusion:
            7. The Court finds that Section 24 of the Public Order Act, to the extent that it requires prior notification rather than permission, is constitutional.
            8. However, the implementation of the Act by the police in this case was unconstitutional.
            
            Orders:
            9. The petition is allowed in part.
            10. The arrest of the Petitioner is declared unconstitutional.
            11. The Respondent shall pay the Petitioner damages of KSh. 500,000.
            
            DATED and DELIVERED at NAIROBI this 30th day of June 2022.
            
            ...................................
            JUDGE OF THE HIGH COURT
            """
        },
        {
            "title": "Republic v. Kamau (2021)",
            "content": """
            REPUBLIC OF KENYA
            IN THE HIGH COURT OF KENYA AT MOMBASA
            CRIMINAL CASE NO. 456 OF 2021
            
            REPUBLIC...................................................................PROSECUTOR
            
            VERSUS
            
            JAMES KAMAU...................................................................ACCUSED
            
            JUDGMENT
            
            Introduction:
            1. The Accused, James Kamau, is charged with the offense of robbery with violence contrary to Section 296(2) of the Penal Code.
            
            Evidence:
            2. The Prosecution called four witnesses who testified that on 5th January 2021, the Accused, armed with a pistol, robbed Mr. David Ochieng of his mobile phone and KSh. 15,000.
            3. The Accused gave a sworn defense denying any involvement in the robbery and providing an alibi.
            
            Analysis:
            4. The Court has carefully considered the evidence adduced by both the Prosecution and the Defense.
            5. The identification of the Accused was made under difficult circumstances as the incident occurred at night.
            6. The Prosecution has not proved beyond reasonable doubt that it was the Accused who committed the robbery.
            
            Conclusion:
            7. The Court finds that the Prosecution has not discharged its burden of proof to the required standard.
            
            Orders:
            8. The Accused is acquitted under Section 215 of the Criminal Procedure Code.
            
            DATED and DELIVERED at MOMBASA this 15th day of December 2021.
            
            ...................................
            JUDGE OF THE HIGH COURT
            """
        },
        {
            "title": "Otieno v. Nairobi County Government (2023)",
            "content": """
            REPUBLIC OF KENYA
            IN THE ENVIRONMENT AND LAND COURT AT NAIROBI
            ELC CASE NO. 789 OF 2023
            
            BETWEEN
            
            MARY OTIENO.................................................................PLAINTIFF
            
            AND
            
            NAIROBI COUNTY GOVERNMENT........................................DEFENDANT
            
            JUDGMENT
            
            Introduction:
            1. The Plaintiff, Mary Otieno, filed this suit on 10th February 2023 seeking declarations that she is the rightful owner of Land Reference No. 12345/67 situated in Westlands, Nairobi.
            
            Background:
            2. The Plaintiff claims to have purchased the suit property from the original allottee in 1995 and has been in occupation since then.
            3. The Defendant claims that the land is public land reserved for a public school.
            
            Issues for Determination:
            4. Whether the Plaintiff is the rightful owner of the suit property.
            5. Whether the suit property is public land.
            
            Analysis:
            6. The Court has examined the documents of title produced by the Plaintiff, including the sale agreement, transfer forms, and certificate of title.
            7. The Defendant has not produced any evidence to show that the land was reserved for a public school.
            
            Findings:
            8. The Court finds that the Plaintiff has established her case on a balance of probabilities.
            9. The title document held by the Plaintiff is valid and was issued by the relevant government authority.
            
            Orders:
            10. The Plaintiff is declared the rightful owner of Land Reference No. 12345/67.
            11. The Defendant is permanently restrained from interfering with the Plaintiff's quiet possession of the suit property.
            12. The Defendant shall bear the costs of this suit.
            
            DATED and DELIVERED at NAIROBI this 20th day of July 2023.
            
            ...................................
            JUDGE OF THE ENVIRONMENT AND LAND COURT
            """
        },
        {
            "title": "Constitution of Kenya - Articles on Rights and Freedoms",
            "content": """
            CONSTITUTION OF KENYA, 2010
            
            PART 2 - RIGHTS AND FUNDAMENTAL FREEDOMS
            
            Article 33 - Freedom of Expression
            
            (1) Every person has the right to freedom of expression, which includes—
                (a) freedom to seek, receive or impart information or ideas;
                (b) freedom of artistic creativity; and
                (c) academic freedom and freedom of scientific research.
            
            (2) The right to freedom of expression does not extend to—
                (a) propaganda for war;
                (b) incitement to violence;
                (c) hate speech; or
                (d) advocacy of hatred that—
                    (i) constitutes ethnic incitement, vilification of others or incitement to cause harm; or
                    (ii) is based on any ground of discrimination specified or contemplated in Article 27(4).
            
            (3) In the exercise of the right to freedom of expression, every person shall respect the rights and reputation of others.
            
            Article 37 - Assembly, Demonstration, Picketing and Petition
            
            Every person has the right, peaceably and unarmed, to assemble, to demonstrate, to picket, and to present petitions to public authorities.
            
            Article 40 - Protection of Right to Property
            
            (1) Subject to Article 65, every person has the right, either individually or in association with others, to acquire and own property of any description; and in any part of Kenya.
            
            (2) Parliament shall not enact a law that permits the State or any person—
                (a) to arbitrarily deprive a person of property of any description or of any interest in, or right over, any property of any description; or
                (b) to limit, or in any way restrict the enjoyment of any right under this Article on the basis of any of the grounds specified or contemplated in Article 27(4).
            
            (3) The State shall not deprive a person of property of any description, or of any interest in, or right over, property of any description, unless the deprivation—
                (a) results from an acquisition of land or an interest in land or a conversion of an interest in land, or title to land, in accordance with Chapter Five; or
                (b) is for a public purpose or in the public interest and is carried out in accordance with this Constitution and any Act of Parliament that—
                    (i) requires prompt payment in full, of just compensation to the person; and
                    (ii) allows any person who has an interest in, or right over, that property a right of access to a court of law.
            """
        },
        {
            "title": "Public Order Act - Section 24",
            "content": """
            PUBLIC ORDER ACT
            
            CHAPTER 56 OF THE LAWS OF KENYA
            
            Section 24 - Regulation of Public Gatherings
            
            (1) No person shall hold a public gathering or a public procession —
                (a) at which the terms of Article 33(2) of the Constitution are contravened; or
                (b) at which any person is incited to the commission of an offence.
            
            (2) Any person intending to convene a public meeting or a public procession shall notify the regulating officer of such intent at least three days but not more than fourteen days before the proposed date of the public meeting or procession.
            
            (3) A notice under subsection (2) shall be in the prescribed form and shall specify—
                (a) the full names and physical address of the organizer of the proposed public meeting or public procession;
                (b) the proposed date of the meeting or procession and the time thereof which shall be between six o'clock in the morning and six o'clock in the afternoon;
                (c) the proposed site of the public meeting or the proposed route in the case of a public procession.
            
            (4) Where a notice under subsection (2) is given in relation to a proposed public meeting or public procession, the regulating officer shall, on being satisfied that the provisions of subsection (3) have been complied with, forthwith issue a receipt thereof to the organizer.
            
            (5) A regulating officer may cancel or prohibit the holding of a public meeting or public procession where—
                (a) a notice of another public meeting or procession on the date, at the time and at the venue proposed has already been received by the regulating officer;
                (b) the public meeting or public procession is intended to be held at a place or building which is unsuitable for that purpose having regard to considerations of public safety or public order; or
                (c) the regulating officer has reasonable grounds to believe that the public meeting or public procession is likely to cause a breach of the peace.
            """
        }
    ]
    
    # Write sample cases to files
    file_paths = []
    for i, case in enumerate(sample_cases):
        filename = f"{output_dir}/doc_{i+1}.txt"
        with open(filename, "w") as f:
            f.write(case["content"])
        print(f"Created sample file: {filename}")
        file_paths.append(filename)
    
    return file_paths

def create_sample_german_law_documents(output_dir="data/german_law"):
    """
    Create sample German legal case files for demonstration purposes.
    """
    print(f"Creating sample German documents in {output_dir}...")
    os.makedirs(output_dir, exist_ok=True)
    
    # Sample German cases
    sample_cases = [
        {
            "title": "BGH Urteil vom 12.05.2021 - VIII ZR 42/20",
            "content": """
            BUNDESGERICHTSHOF
            IM NAMEN DES VOLKES
            URTEIL
            VIII ZR 42/20
            
            In dem Rechtsstreit
            
            [Anonymisiert] - Kläger und Revisionskläger
            
            gegen
            
            [Anonymisiert] - Beklagte und Revisionsbeklagte
            
            hat der VIII. Zivilsenat des Bundesgerichtshofs auf die mündliche Verhandlung
            vom 12. Mai 2021 durch [Richtername] als Vorsitzenden, die Richter [Namen]
            und die Richterinnen [Namen] für Recht erkannt:
            
            Tenor:
            Auf die Revision des Klägers wird das Urteil des Landgerichts München I
            vom 21. Januar 2020 aufgehoben. Die Sache wird zur neuen Verhandlung
            und Entscheidung an das Berufungsgericht zurückverwiesen.
            
            Die Kosten des Revisionsverfahrens hat die Beklagte zu tragen.
            
            Tatbestand:
            Der Kläger mietete von der Beklagten eine Wohnung in München. Nach Beendigung
            des Mietverhältnisses verlangt er die Rückzahlung der Mietkaution. Die Beklagte
            verweigert dies unter Hinweis auf angebliche Schäden in der Wohnung.
            
            Entscheidungsgründe:
            Die Revision des Klägers ist begründet. Das Berufungsgericht hat verkannt,
            dass der Vermieter die Beweislast für die behaupteten Schäden trägt. Die bloße
            Behauptung von Schäden reicht nicht aus, um die Kaution einzubehalten.
            
            Das Berufungsgericht wird die erforderlichen Feststellungen nachzuholen haben.
            """
        },
        {
            "title": "BVerfG Beschluss vom 03.03.2022 - 1 BvR 2564/21",
            "content": """
            BUNDESVERFASSUNGSGERICHT
            - 1 BvR 2564/21 -
            
            In dem Verfahren über die Verfassungsbeschwerde
            
            des Herrn [Anonymisiert],
            
            gegen
            
            den Beschluss des Oberlandesgerichts Frankfurt am Main vom 14. Oktober 2021
            
            hat die 3. Kammer des Ersten Senats des Bundesverfassungsgerichts durch
            die Richter [Namen] am 3. März 2022 einstimmig beschlossen:
            
            Die Verfassungsbeschwerde wird nicht zur Entscheidung angenommen.
            
            Gründe:
            Die Verfassungsbeschwerde wird nicht zur Entscheidung angenommen, weil die
            Annahmevoraussetzungen des § 93a Abs. 2 BVerfGG nicht vorliegen. Die
            Verfassungsbeschwerde hat keine grundsätzliche verfassungsrechtliche Bedeutung.
            
            Die angegriffene Entscheidung verletzt den Beschwerdeführer nicht in seinem
            Grundrecht aus Art. 5 Abs. 1 Satz 1 GG. Das Oberlandesgericht hat die Bedeutung
            und Tragweite der Meinungsfreiheit bei der Auslegung und Anwendung der
            einschlägigen Vorschriften in verfassungsrechtlich nicht zu beanstandender
            Weise berücksichtigt.
            """
        },
        {
            "title": "BGH Urteil vom 17.06.2021 - III ZR 19/20 - Cum-Ex",
            "content": """
            BUNDESGERICHTSHOF
            IM NAMEN DES VOLKES
            URTEIL
            III ZR 19/20
            
            In dem Rechtsstreit
            
            [Anonymisiert] - Kläger und Revisionsbeklagter
            
            gegen
            
            [Anonymisiert] - Beklagte und Revisionsklägerin
            
            hat der III. Zivilsenat des Bundesgerichtshofs auf die mündliche Verhandlung
            vom 17. Juni 2021 durch die Richter [Namen] für Recht erkannt:
            
            Tenor:
            Die Revision der Beklagten gegen das Urteil des Oberlandesgerichts Frankfurt am Main
            vom 12. Dezember 2019 wird zurückgewiesen.
            
            Die Beklagte trägt die Kosten des Revisionsverfahrens.
            
            Tatbestand:
            Der Kläger begehrt Schadensersatz wegen fehlerhafter Anlageberatung im Zusammenhang
            mit Cum-Ex-Geschäften durch die beklagte Bank. Er hatte auf Anraten der Bank in
            einen Fonds investiert, der mit Cum-Ex-Transaktionen Steuervorteile erzielen wollte.
            
            Entscheidungsgründe:
            Die Beklagte hat ihre Beratungspflichten verletzt, indem sie den Kläger nicht
            über die steuerrechtlichen Risiken der Cum-Ex-Geschäfte aufgeklärt hat. Bereits
            zum Zeitpunkt der Beratung bestanden erhebliche Zweifel an der steuerrechtlichen
            Zulässigkeit solcher Transaktionen.
            """
        },
        {
            "title": "Grundgesetz - Artikel zur Meinungsfreiheit",
            "content": """
            GRUNDGESETZ FÜR DIE BUNDESREPUBLIK DEUTSCHLAND
            
            Artikel 5
            
            (1) Jeder hat das Recht, seine Meinung in Wort, Schrift und Bild frei zu
            äußern und zu verbreiten und sich aus allgemein zugänglichen Quellen
            ungehindert zu unterrichten. Die Pressefreiheit und die Freiheit der
            Berichterstattung durch Rundfunk und Film werden gewährleistet.
            Eine Zensur findet nicht statt.
            
            (2) Diese Rechte finden ihre Schranken in den Vorschriften der allgemeinen
            Gesetze, den gesetzlichen Bestimmungen zum Schutze der Jugend und in dem
            Recht der persönlichen Ehre.
            
            (3) Kunst und Wissenschaft, Forschung und Lehre sind frei. Die Freiheit
            der Lehre entbindet nicht von der Treue zur Verfassung.
            """
        },
        {
            "title": "BGB - Paragraphen zum Mietrecht",
            "content": """
            BÜRGERLICHES GESETZBUCH (BGB)
            
            § 535 Inhalt und Hauptpflichten des Mietvertrags
            
            (1) Durch den Mietvertrag wird der Vermieter verpflichtet, dem Mieter den
            Gebrauch der Mietsache während der Mietzeit zu gewähren. Der Vermieter hat
            die Mietsache dem Mieter in einem zum vertragsgemäßen Gebrauch geeigneten
            Zustand zu überlassen und sie während der Mietzeit in diesem Zustand zu
            erhalten. Er hat die auf der Mietsache ruhenden Lasten zu tragen.
            
            (2) Der Mieter ist verpflichtet, dem Vermieter die vereinbarte Miete zu entrichten.
            
            § 546 Rückgabepflicht des Mieters
            
            (1) Der Mieter ist verpflichtet, die Mietsache nach Beendigung des
            Mietverhältnisses zurückzugeben.
            
            (2) Hat der Mieter den Gebrauch der Mietsache einem Dritten überlassen, so
            kann der Vermieter die Sache nach Beendigung des Mietverhältnisses auch von
            dem Dritten zurückfordern.
            
            § 551 Begrenzung und Anlage von Mietsicherheiten
            
            (1) Hat der Mieter dem Vermieter für die Erfüllung seiner Pflichten Sicherheit
            zu leisten, so darf diese vorbehaltlich des Absatzes 3 Satz 4 höchstens das
            Dreifache der auf einen Monat entfallenden Miete ohne die als Pauschale oder
            als Vorauszahlung ausgewiesenen Betriebskosten betragen.
            
            (2) Ist als Sicherheit eine Geldsumme bereitzustellen, so ist der Mieter zu
            drei gleichen monatlichen Teilzahlungen berechtigt. Die erste Teilzahlung ist
            zu Beginn des Mietverhältnisses fällig.
            """
        }
    ]
    
    # Write sample cases to files
    file_paths = []
    for i, case in enumerate(sample_cases):
        filename = f"{output_dir}/german_doc_{i+1}.txt"
        with open(filename, "w") as f:
            f.write(case["content"])
        print(f"Created sample file: {filename}")
        file_paths.append(filename)
    
    return file_paths

# Process documents
def load_documents(file_paths, jurisdiction="kenya"):
    """
    Load and process documents into Document objects for vector store.
    Now includes jurisdiction metadata.
    """
    documents = []
    
    for file_path in file_paths:
        with open(file_path, 'r') as f:
            content = f.read()
            
        # Create document with metadata including jurisdiction
        documents.append(
            Document(
                page_content=content,
                metadata={
                    "source": file_path,
                    "filename": os.path.basename(file_path),
                    "jurisdiction": jurisdiction
                }
            )
        )
    
    print(f"Loaded {len(documents)} {jurisdiction} documents")
    return documents

# Split documents into chunks
def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    """
    Split documents into chunks for embedding.
    """
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    
    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", ".", " ", ""]
    )
    
    chunks = text_splitter.split_documents(documents)
    
    print(f"Split into {len(chunks)} chunks")
    return chunks

# Simplified Vector Store Implementation (avoid dependency issues)
class SimpleVectorStore:
    """
    A simplified vector store implementation that doesn't rely on external embedding libraries.
    Uses Google Generative AI embedding API directly.
    """
    def __init__(self, documents, cache_dir="embeddings/kenya_law", jurisdiction="kenya"):
        self.documents = documents
        self.cache_dir = cache_dir
        self.embeddings = {}
        self.jurisdiction = jurisdiction
        os.makedirs(cache_dir, exist_ok=True)
        self._compute_embeddings()
        
    def _compute_embeddings(self):
        """Compute embeddings for all documents using Google's API."""
        print(f"Computing embeddings for {self.jurisdiction} documents...")
        genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
        
        for i, doc in enumerate(self.documents):
            # Use text chunk as embedding key (for simplicity)
            key = f"doc_{i}"
            text = doc.page_content
            
            try:
                # Use Google's embedding model
                result = genai.embed_content(
                    model="models/embedding-001",
                    content=text[:8000],  # API has text length limits
                    task_type="retrieval_query"
                )
                self.embeddings[key] = {
                    "vector": result["embedding"],
                    "document": doc
                }
                
                # Simple progress indicator
                if (i + 1) % 5 == 0 or i == len(self.documents) - 1:
                    print(f"Embedded {i + 1}/{len(self.documents)} documents")
                    
            except Exception as e:
                print(f"Error embedding document {i}: {e}")
        
        print(f"Created vector store with {len(self.embeddings)} document embeddings for {self.jurisdiction} jurisdiction")
        
    def persist(self):
        """Save embeddings to disk."""
        # Save document info separately from vectors (which can be large)
        doc_info = {k: {
            "source": v["document"].metadata.get("source", ""),
            "filename": v["document"].metadata.get("filename", ""),
            "jurisdiction": v["document"].metadata.get("jurisdiction", self.jurisdiction)
        } for k, v in self.embeddings.items()}
        
        with open(f"{self.cache_dir}/doc_info.json", "w") as f:
            json.dump(doc_info, f)
            
        print(f"Vector store metadata saved to {self.cache_dir}")
        
    def similarity_search(self, query, k=3):
        """Search for similar documents using dot product similarity."""
        # Get query embedding
        try:
            result = genai.embed_content(
                model="models/embedding-001",
                content=query,
                task_type="retrieval_query"
            )
            query_embedding = result["embedding"]
            
            # Calculate similarity scores
            similarities = {}
            for key, data in self.embeddings.items():
                doc_embedding = data["vector"]
                
                # Simple dot product similarity
                similarity = sum(q * d for q, d in zip(query_embedding, doc_embedding))
                similarities[key] = similarity
            
            # Get top k results
            top_keys = sorted(similarities.keys(), key=lambda k: similarities[k], reverse=True)[:k]
            results = [self.embeddings[key]["document"] for key in top_keys]
            
            return results
            
        except Exception as e:
            print(f"Search error: {e}")
            return []
    
    def similarity_search_with_score(self, query, k=3):
        """Search for similar documents and return (doc, score) pairs."""
        # Get query embedding
        try:
            result = genai.embed_content(
                model="models/embedding-001",
                content=query,
                task_type="retrieval_query"
            )
            query_embedding = result["embedding"]
            
            # Calculate similarity scores
            similarities = {}
            for key, data in self.embeddings.items():
                doc_embedding = data["vector"]
                
                # Simple dot product similarity
                similarity = sum(q * d for q, d in zip(query_embedding, doc_embedding))
                similarities[key] = similarity
            
            # Get top k results
            top_keys = sorted(similarities.keys(), key=lambda k: similarities[k], reverse=True)[:k]
            results = [(self.embeddings[key]["document"], similarities[key]) for key in top_keys]
            
            return results
            
        except Exception as e:
            print(f"Search error: {e}")
            return []
    
    def as_retriever(self, search_kwargs=None):
        """Return a retriever object for use with LangChain."""
        search_kwargs = search_kwargs or {}
        
        class SimpleRetriever:
            def __init__(self, vector_store, search_kwargs):
                self.vector_store = vector_store
                self.search_kwargs = search_kwargs
                
            def get_relevant_documents(self, query):
                k = self.search_kwargs.get("k", 3)
                return self.vector_store.similarity_search(query, k=k)
        
        return SimpleRetriever(self, search_kwargs)

def create_document_stores():
    """
    Create and load both Kenyan and German legal document stores
    """
    # Update context cache structure to support multiple jurisdictions
    context_cache["vector_stores"] = {
        "kenya": None,
        "germany": None
    }
    context_cache["document_index"] = {
        "kenya": {},
        "germany": {}
    }
    
    # Create sample Kenyan documents
    kenyan_file_paths = create_sample_kenya_law_documents()
    
    # Create sample German documents
    german_file_paths = create_sample_german_law_documents()
    
    # Load Kenyan documents
    kenyan_documents = load_documents(kenyan_file_paths, jurisdiction="kenya")
    kenyan_document_chunks = split_documents(kenyan_documents)
    kenyan_vector_store = SimpleVectorStore(kenyan_document_chunks, cache_dir="embeddings/kenya_law", jurisdiction="kenya")
    kenyan_vector_store.persist()
    
    # Load German documents
    german_documents = load_documents(german_file_paths, jurisdiction="germany")
    german_document_chunks = split_documents(german_documents)
    german_vector_store = SimpleVectorStore(german_document_chunks, cache_dir="embeddings/german_law", jurisdiction="germany")
    german_vector_store.persist()
    
    # Add to context
    context_cache["vector_stores"]["kenya"] = kenyan_vector_store
    context_cache["vector_stores"]["germany"] = german_vector_store
    
    # Build document indexes for both jurisdictions
    for doc in kenyan_documents:
        doc_id = doc.metadata["filename"]
        context_cache["document_index"]["kenya"][doc_id] = {
            "source": doc.metadata["source"],
            "title": doc.metadata["filename"].replace(".txt", "").replace("doc_", "Case "),
            "jurisdiction": "kenya"
        }
        
    for doc in german_documents:
        doc_id = doc.metadata["filename"]
        context_cache["document_index"]["germany"][doc_id] = {
            "source": doc.metadata["source"],
            "title": doc.metadata["filename"].replace(".txt", "").replace("german_doc_", "German Case "),
            "jurisdiction": "germany"
        }
    
    print("Vector stores and document indexes created for both Kenyan and German jurisdictions.")
    
    return kenyan_documents, german_documents

# Initialize document stores for both jurisdictions
kenyan_documents, german_documents = create_document_stores()

Creating sample Kenyan documents in data/raw...
Created sample file: data/raw/doc_1.txt
Created sample file: data/raw/doc_2.txt
Created sample file: data/raw/doc_3.txt
Created sample file: data/raw/doc_4.txt
Created sample file: data/raw/doc_5.txt
Creating sample German documents in data/german_law...
Created sample file: data/german_law/german_doc_1.txt
Created sample file: data/german_law/german_doc_2.txt
Created sample file: data/german_law/german_doc_3.txt
Created sample file: data/german_law/german_doc_4.txt
Created sample file: data/german_law/german_doc_5.txt
Loaded 5 kenya documents
Split into 15 chunks
Computing embeddings for kenya documents...
Embedded 5/15 documents
Embedded 10/15 documents
Embedded 15/15 documents
Created vector store with 15 document embeddings for kenya jurisdiction
Vector store metadata saved to embeddings/kenya_law
Loaded 5 germany documents
Split into 9 chunks
Computing embeddings for germany documents...
Embedded 5/9 documents
Embedded 9/9 documents


# Step 5: Define Monitoring & RAG Functions

Search Function (Placeholder/Simulation): Create a function to simulate searching for real-time info. Crucially, this needs to be replaced with a real API call (e.g., Google Custom Search JSON API, SerpApi, or specific APIs for flights/weather).

Analysis Function (RAG): Create a function that takes search results and the relevant part of the itinerary, then asks the Gemini model to analyze if there's a conflict or relevant update.

In [5]:
# === SECTION 5: DEFINE SEARCH AND GROUNDING FUNCTIONS ===

def search_legal_information(query_type, search_terms, jurisdiction="kenya", max_results=5):
    """
    Search for legal information using Google Search API or vector store.
    Now includes jurisdiction selection.
    
    Args:
        query_type: Type of query (case_law, statute, general)
        search_terms: Terms to search for
        jurisdiction: 'kenya' or 'germany'
        max_results: Maximum number of results to return
        
    Returns:
        Dictionary with search results
    """
    print(f"\n--- Performing Legal Search in {jurisdiction.title()} Law ---")
    print(f"Query Type: {query_type}, Search Terms: {search_terms}")
    
    # Check if results are in cache
    cache_key = f"{jurisdiction}:{query_type}:{search_terms}"
    if cache_key in context_cache["search_results_cache"]:
        print("Using cached search results.")
        return context_cache["search_results_cache"][cache_key]
    
    # Try vector store search first for all query types
    vector_results = []
    try:
        # Get the appropriate vector store based on jurisdiction
        vector_store = context_cache["vector_stores"].get(jurisdiction)
        if vector_store:
            retriever = vector_store.as_retriever(search_kwargs={"k": max_results})
            vector_docs = retriever.get_relevant_documents(search_terms)
            
            for doc in vector_docs:
                snippet = doc.page_content[:300] + "..." if len(doc.page_content) > 300 else doc.page_content
                vector_results.append({
                    "title": context_cache["document_index"][jurisdiction].get(os.path.basename(doc.metadata["source"]), {}).get("title", "Unknown Document"),
                    "snippet": snippet,
                    "source": doc.metadata["source"],
                    "jurisdiction": jurisdiction
                })
        else:
            print(f"Vector store for {jurisdiction} not available")
    except Exception as e:
        print(f"Vector search error: {e}")
    
    # If we have Google Search credentials and need external information, use Google Search
    google_api_key = os.environ.get('GOOGLE_API_KEY')
    google_cse_id = os.environ.get('GOOGLE_CSE_ID')
    
    if google_api_key and google_cse_id and query_type != "internal_only":
        try:
            # Construct query based on type and jurisdiction
            if jurisdiction == "kenya":
                if query_type == "case_law":
                    query = f"Kenya Law Reports {search_terms} court judgment"
                elif query_type == "statute":
                    query = f"Kenya {search_terms} law statute regulation"
                else:
                    query = f"Kenya legal {search_terms}"
            else:  # Germany
                if query_type == "case_law":
                    query = f"German Bundesgerichtshof BGH {search_terms} Urteil"
                elif query_type == "statute":
                    query = f"German BGB {search_terms} Gesetz Paragraph"
                else:
                    query = f"German legal {search_terms} Recht"
            
            # Call Google Search API
            search_url = "https://www.googleapis.com/customsearch/v1"
            params = {
                'key': google_api_key,
                'cx': google_cse_id,
                'q': query,
                'num': max_results
            }
            
            response = requests.get(search_url, params=params, timeout=10)
            response.raise_for_status()
            search_results = response.json()
            
            # Extract results
            google_results = []
            if "items" in search_results:
                for item in search_results["items"]:
                    google_results.append({
                        "title": item.get("title", ""),
                        "link": item.get("link", ""),
                        "snippet": item.get("snippet", ""),
                        "jurisdiction": jurisdiction
                    })
            
            # Combine results, prioritizing vector results first
            combined_results = vector_results + [r for r in google_results if not any(vr["title"] == r["title"] for vr in vector_results)]
            
            # Cache results
            context_cache["search_results_cache"][cache_key] = {
                "source": "Combined (Vector DB + Google Search)",
                "query": search_terms,
                "jurisdiction": jurisdiction,
                "results": combined_results[:max_results]
            }
            return context_cache["search_results_cache"][cache_key]
            
        except Exception as e:
            print(f"Google Search error: {e}")
            # Fall back to vector results only
    
    # Return vector results if Google Search failed or wasn't attempted
    results = {
        "source": "Vector DB Only",
        "query": search_terms,
        "jurisdiction": jurisdiction,
        "results": vector_results
    }
    
    # Cache results
    context_cache["search_results_cache"][cache_key] = results
    return results

def ground_legal_analysis(query, search_results, context):
    """
    Uses RAG approach to ground legal analysis in search results.
    """
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-pro-exp-03-25",
        google_api_key=os.environ["GOOGLE_API_KEY"],
        temperature=0.1
    )
    
    prompt = f"""
    You are a Kenyan legal expert specializing in legal analysis and research.
    
    Provide a well-grounded legal analysis on the following query, using ONLY the provided search results as evidence.
    If the search results don't contain sufficient information to answer the query fully, acknowledge the limitations.
    
    User Preferences: {json.dumps(context.get("user_preferences", {}))}
    
    Query: {query}
    
    Search Results:
    {json.dumps(search_results, indent=2)}
    
    Your grounded legal analysis:
    """
    
    try:
        response = llm.invoke(prompt)
        analysis = response.content
        
        # Add to conversation history
        context["conversation_history"].append({"role": "user", "parts": f"Analyze: {query}"})
        context["conversation_history"].append({"role": "model", "parts": analysis})
        
        return analysis
    except Exception as e:
        print(f"ERROR: Grounding analysis failed: {e}")
        return f"Analysis failed due to technical error: {str(e)}"

# Step 6: Define Function Calling for Alternatives/Availability

Define Python Functions: Create stub functions that would perform actions like checking museum hours, finding restaurants, or checking tour availability. These won't actually do the booking/checking in this example, but they define the structure for the AI to call.

Register Functions with Gemini: Tell the Gemini model about these available tools.

Create Suggestion Function: A function that, upon detecting a disruption, asks Gemini to suggest alternatives, enabling it to use the defined functions if needed.

In [6]:
# === SECTION 6: DEFINE LEGAL ASSISTANT FUNCTIONS ===

def analyze_case_law(case_text, analysis_type="comprehensive", jurisdiction="kenya"):
    """
    Analyze a legal case with Gemini Pro and return structured analysis.
    Now supports both Kenyan and German cases.
    """
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-pro-exp-03-25",
        google_api_key=os.environ["GOOGLE_API_KEY"],
        temperature=0.1
    )
    
    # Different analysis templates based on jurisdiction and type
    kenyan_templates = {
        "brief": """
            Provide a brief summary (3-5 sentences) of this Kenyan legal case,
            highlighting only the most essential facts, issue, and holding.
        """,
        
        "comprehensive": """
            Provide a comprehensive analysis of this Kenyan legal case in the following structured format:
            
            # CASE SUMMARY
            
            ## Citation
            [Extract or construct a proper citation]
            
            ## Court
            [Name of the court]
            
            ## Parties
            [List the parties involved]
            
            ## Date
            [Date of judgment]
            
            ## Facts
            [Key facts of the case in 3-5 bullet points]
            
            ## Issues
            [Main legal issues presented as questions]
            
            ## Holding
            [Court's final decision on each issue]
            
            ## Legal Principles
            [Key legal principles established or applied]
            
            ## Reasoning
            [Court's reasoning in reaching the decision]
            
            ## Significance
            [Brief explanation of case significance in Kenyan law]
        """,
        
        "legal_principles": """
            Extract and explain the key legal principles established or applied in this Kenyan case.
            For each principle:
            1. State the principle clearly
            2. Explain how the court applied it in this case
            3. Note how this might impact future cases
        """
    }
    
    german_templates = {
        "brief": """
            Provide a brief summary (3-5 sentences) of this German legal case,
            highlighting only the most essential facts, issue, and holding.
            Use standard German legal terminology where appropriate.
        """,
        
        "comprehensive": """
            Provide a comprehensive analysis of this German legal case in the following structured format:
            
            # CASE SUMMARY
            
            ## Citation
            [Extract or construct a proper citation using German legal format]
            
            ## Court
            [Name of the court - use official German court name with English translation]
            
            ## Parties
            [List the parties involved]
            
            ## Date
            [Date of judgment]
            
            ## Facts (Sachverhalt)
            [Key facts of the case in 3-5 bullet points]
            
            ## Issues (Rechtsfragen)
            [Main legal issues presented as questions]
            
            ## Holding (Tenor/Entscheidungsformel)
            [Court's final decision on each issue]
            
            ## Legal Principles (Rechtsgrundsätze)
            [Key legal principles established or applied]
            
            ## Reasoning (Entscheidungsgründe)
            [Court's reasoning in reaching the decision]
            
            ## Significance
            [Brief explanation of case significance in German law]
        """,
        
        "legal_principles": """
            Extract and explain the key legal principles (Rechtsgrundsätze) established or applied in this German case.
            For each principle:
            1. State the principle clearly in both German legal terminology and English translation
            2. Explain how the court applied it in this case
            3. Note how this might impact future cases in the German legal system
        """
    }
    
    # Select template based on jurisdiction
    templates = german_templates if jurisdiction == "germany" else kenyan_templates
    template = templates.get(analysis_type, templates["comprehensive"])
    
    # Create appropriate prompt based on jurisdiction
    jurisdiction_name = "German" if jurisdiction == "germany" else "Kenyan"
    prompt = f"""
    You are a {jurisdiction_name} legal expert specialized in case analysis.
    
    {template}
    
    CASE TEXT:
    {case_text}
    """
    
    try:
        response = llm.invoke(prompt)
        return response.content
    except Exception as e:
        print(f"ERROR: Case analysis failed: {e}")
        return f"Analysis failed: {str(e)}"

def generate_legal_document(document_type, details, jurisdiction="kenya"):
    """
    Generate a legal document based on provided details.
    Now supports both Kenyan and German legal documents.
    """
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-pro-exp-03-25",
        google_api_key=os.environ["GOOGLE_API_KEY"],
        temperature=0.2
    )
    
    # Kenyan document templates
    kenyan_templates = {
        "petition": """
            Create a formal petition to be filed in a Kenyan court. Include:
            
            1. Proper heading with court name, case number placeholder, and parties
            2. Introduction stating the legal basis for the petition
            3. Statement of facts based on the provided details
            4. Legal arguments with appropriate citations to Kenyan statutes and cases
            5. Prayer for relief (specific requests to the court)
            6. Closing with signature blocks for advocate and petitioner
            
            Follow Kenyan legal formatting conventions throughout.
        """,
        
        "legal_memo": """
            Create a formal legal memorandum analyzing a legal issue under Kenyan law. Include:
            
            1. Heading with TO, FROM, DATE, and SUBJECT lines
            2. Introduction stating the legal question
            3. Brief statement of relevant facts
            4. Discussion of applicable law with citations to statutes and cases
            5. Application of law to facts
            6. Conclusion with clear legal opinion
            
            Use formal legal writing style appropriate for Kenyan legal practice.
        """,
        
        "affidavit": """
            Create a formal affidavit for use in a Kenyan court. Include:
            
            1. Proper heading with court name, case number, and parties
            2. Introduction identifying the deponent
            3. Numbered paragraphs with factual statements
            4. Verification clause
            5. Jurat (to be signed before a Commissioner for Oaths)
            
            Follow Kenyan legal formatting conventions throughout.
        """
    }
    
    # German document templates
    german_templates = {
        "klage": """
            Create a formal lawsuit filing (Klageschrift) for a German court. Include:
            
            1. Proper heading with court name (Amtsgericht/Landgericht) and parties
            2. Introduction and application for relief (Antrag)
            3. Statement of facts (Sachverhalt) based on the provided details
            4. Legal arguments (Rechtliche Würdigung) with citations to German law (BGB, etc.)
            5. Conclusion with specific requests (Anträge)
            6. Closing with signature blocks for attorney (Rechtsanwalt)
            
            Follow German legal formatting conventions throughout and use appropriate German legal terminology.
        """,
        
        "rechtsgutachten": """
            Create a formal legal opinion (Rechtsgutachten) analyzing a legal issue under German law. Include:
            
            1. Introduction (Sachverhalt) with a statement of facts
            2. Legal question(s) to be addressed (Rechtsfrage)
            3. Examination (Rechtliche Prüfung) with structured analysis using German legal methodology
            4. Analysis of applicable laws with citations to relevant provisions of German statutes and cases
            5. Conclusion (Ergebnis) with clear legal opinion
            
            Use formal German legal writing style with appropriate terminology and organization.
        """,
        
        "eidesstattliche_versicherung": """
            Create a formal statutory declaration (Eidesstattliche Versicherung) for use in a German legal proceeding. Include:
            
            1. Proper heading with court information if applicable
            2. Introduction identifying the declarant with full name and address
            3. Numbered paragraphs with factual statements
            4. Closing declaration that affirms awareness of criminal penalties for false statements
            5. Date and signature line
            
            Follow German legal conventions throughout and use standard German legal phrases.
        """
    }
    
    # Select appropriate templates based on jurisdiction
    if jurisdiction == "germany":
        # Map document types for German jurisdiction
        document_mapping = {
            "petition": "klage",
            "legal_memo": "rechtsgutachten",
            "affidavit": "eidesstattliche_versicherung"
        }
        # Use the mapped document type or the original if no mapping exists
        german_doc_type = document_mapping.get(document_type, document_type)
        template = german_templates.get(german_doc_type, german_templates["rechtsgutachten"])
    else:
        template = kenyan_templates.get(document_type, kenyan_templates["legal_memo"])
    
    details_text = "\n".join([f"{key}: {value}" for key, value in details.items()])
    
    jurisdiction_name = "German" if jurisdiction == "germany" else "Kenyan"
    prompt = f"""
    You are a {jurisdiction_name} legal expert specializing in drafting professional legal documents.
    
    {template}
    
    DETAILS:
    {details_text}
    """
    
    try:
        response = llm.invoke(prompt)
        return response.content
    except Exception as e:
        print(f"ERROR: Document generation failed: {e}")
        return f"Document generation failed: {str(e)}"

# Define the function declarations using a compatible format for the Google GenAI library
# Updated to include jurisdiction parameter
search_legal_info_schema = {
    "type": "object",
    "properties": {
        "query_type": {
            "type": "string", 
            "description": "Type of legal query: 'case_law', 'statute', or 'general'.",
            "enum": ["case_law", "statute", "general", "internal_only"]
        },
        "search_terms": {
            "type": "string", 
            "description": "Terms to search for, should be specific and relevant to the legal query."
        },
        "jurisdiction": {
            "type": "string",
            "description": "Legal jurisdiction to search in: 'kenya' or 'germany'.",
            "enum": ["kenya", "germany"],
            "default": "kenya"
        },
        "max_results": {
            "type": "integer",
            "description": "Maximum number of results to return (1-10).",
            "default": 5
        }
    },
    "required": ["query_type", "search_terms"]
}

analyze_case_schema = {
    "type": "object",
    "properties": {
        "case_text": {
            "type": "string", 
            "description": "The full text of the legal case to analyze."
        },
        "analysis_type": {
            "type": "string", 
            "description": "Type of analysis to perform.",
            "enum": ["brief", "comprehensive", "legal_principles"],
            "default": "comprehensive"
        },
        "jurisdiction": {
            "type": "string",
            "description": "Legal jurisdiction of the case: 'kenya' or 'germany'.",
            "enum": ["kenya", "germany"],
            "default": "kenya"
        }
    },
    "required": ["case_text"]
}

generate_document_schema = {
    "type": "object",
    "properties": {
        "document_type": {
            "type": "string", 
            "description": "Type of legal document to generate.",
            "enum": ["petition", "legal_memo", "affidavit", "klage", "rechtsgutachten", "eidesstattliche_versicherung"]
        },
        "details": {
            "type": "object", 
            "description": "Details needed for the document (client name, issue, facts, etc.)."
        },
        "jurisdiction": {
            "type": "string",
            "description": "Legal jurisdiction for the document: 'kenya' or 'germany'.",
            "enum": ["kenya", "germany"],
            "default": "kenya"
        }
    },
    "required": ["document_type", "details"]
}

# Create tools configuration that works with the current library version
tools = [
    {
        "function_declarations": [
            {
                "name": "search_legal_information",
                "description": "Searches for legal information using legal databases and Google Search for Kenya or Germany.",
                "parameters": search_legal_info_schema
            },
            {
                "name": "analyze_case_law",
                "description": "Analyzes a legal case from Kenya or Germany and provides structured analysis.",
                "parameters": analyze_case_schema
            },
            {
                "name": "generate_legal_document",
                "description": "Generates a legal document based on provided details for Kenya or Germany.",
                "parameters": generate_document_schema
            }
        ]
    }
]

# Make functions available to the model
available_functions = {
    "search_legal_information": search_legal_information,
    "analyze_case_law": analyze_case_law,
    "generate_legal_document": generate_legal_document
}

**Implement Context Caching (Integrated)**

Context caching is already implicitly happening by:

Storing user preferences in context_cache["user_preferences"].
Storing the current_itinerary in context_cache["current_itinerary"].
Appending interactions (user requests, AI responses, analysis results) to context_cache["conversation_history"].
When calling the Gemini model (e.g., in analyze_information_for_disruption or suggest_and_evaluate_alternatives), you can pass relevant parts of this history or the full context to maintain continuity. Note: Be mindful of token limits when passing extensive history.

# Step 7: Define HTML Formatting Function

In [7]:
# === SECTION 7: CREATE LEGAL RESEARCH FUNCTION (SIMPLIFIED) ===

# Define a simple legal research function without complex LangChain dependencies
def setup_legal_research():
    """
    Creates a minimal legal research function using just the necessary components.
    Now supports both Kenyan and German jurisdictions.
    """
    # Get the Gemini model configured in Section 2
    global model
    
    def perform_legal_research(query, jurisdiction="kenya"):
        """
        Performs legal research on a query using vector store and Gemini model.
        
        Args:
            query: The legal question or research query
            jurisdiction: 'kenya' or 'germany'
        
        Returns:
            Formatted legal research response
        """
        print(f"Researching in {jurisdiction.upper()} jurisdiction: {query}")
        
        # Step 1: Search internal vector store for the specified jurisdiction
        internal_results = []
        try:
            # Get results from appropriate vector store based on jurisdiction
            vector_store = context_cache["vector_stores"].get(jurisdiction)
            if vector_store:
                vector_results = vector_store.similarity_search(query, k=3)
                for doc in vector_results:
                    internal_results.append({
                        "content": doc.page_content[:1000],
                        "source": doc.metadata.get("source", "Unknown"),
                        "jurisdiction": doc.metadata.get("jurisdiction", jurisdiction)
                    })
                print(f"Found {len(internal_results)} internal {jurisdiction} documents")
            else:
                print(f"Vector store for {jurisdiction} not available")
        except Exception as e:
            print(f"Error searching vector store: {e}")
        
        # Step 2: Search external sources with jurisdiction focus
        external_results = []
        try:
            search_result = search_legal_information("general", query, jurisdiction=jurisdiction)
            if "results" in search_result:
                external_results = search_result["results"]
                print(f"Found {len(external_results)} external results for {jurisdiction}")
            else:
                print(f"No external results found for {jurisdiction}")
        except Exception as e:
            print(f"Error in external search: {e}")
        
        # Step 3: Format context for the model
        context_text = f"{jurisdiction.upper()} INTERNAL DOCUMENTS:\n"
        if internal_results:
            for i, doc in enumerate(internal_results):
                context_text += f"\n[Document {i+1}]\n"
                context_text += f"Source: {doc['source']}\n"
                context_text += f"{doc['content']}\n"
        else:
            context_text += f"No internal {jurisdiction} documents found.\n"
        
        context_text += f"\n{jurisdiction.upper()} EXTERNAL SOURCES:\n"
        if external_results:
            for i, result in enumerate(external_results):
                context_text += f"\n[Result {i+1}]\n"
                context_text += f"Title: {result.get('title', 'Untitled')}\n"
                context_text += f"Content: {result.get('snippet', 'No content')}\n"
        else:
            context_text += f"No external {jurisdiction} sources found.\n"
        
        # Step 4: Generate response using Gemini model with jurisdiction-specific prompting
        if jurisdiction == "germany":
            prompt = f"""
            You are an expert German legal researcher specializing in German law (Deutsches Recht).
            Provide a comprehensive legal analysis on the following query using ONLY the information 
            from the provided German legal sources. If the sources don't contain enough information,
            acknowledge the limitations.
            
            QUERY: {query}
            
            SOURCES:
            {context_text}
            
            Provide a structured analysis with:
            1. Summary of the legal issue (Zusammenfassung des Rechtsproblems)
            2. Relevant German laws and statutes (Einschlägige Gesetze)
            3. Key German case precedents (Relevante Rechtsprechung)
            4. Application to the query (Rechtliche Würdigung)
            5. Conclusion (Ergebnis)
            
            Use appropriate German legal terminology where relevant, with English translations.
            """
        else:  # Kenya
            prompt = f"""
            You are an expert Kenyan legal researcher. Provide a comprehensive legal analysis 
            on the following query, using ONLY the information from the provided Kenyan legal sources. 
            If the sources don't contain enough information, acknowledge the limitations.
            
            QUERY: {query}
            
            SOURCES:
            {context_text}
            
            Provide a structured analysis with:
            1. Summary of the legal issue
            2. Relevant Kenyan laws and statutes
            3. Key Kenyan case precedents
            4. Application to the query
            5. Conclusion
            """
        
        try:
            response = model.generate_content(prompt)
            return response.text
        except Exception as e:
            print(f"Error generating response: {e}")
            return f"Failed to generate legal analysis due to an error: {str(e)}"
    
    return perform_legal_research

# Initialize the legal research function
try:
    legal_research_function = setup_legal_research()
    print("Legal research function created successfully.")
except Exception as e:
    print(f"Error creating legal research function: {e}")
    traceback.print_exc()
    legal_research_function = None

# Function to run legal research with error handling
def run_legal_research(query, jurisdiction="kenya"):
    """
    Run legal research with error handling.
    
    Args:
        query: The legal question or research query
        jurisdiction: 'kenya' or 'germany'
    """
    if legal_research_function is None:
        return "Legal research function is not available."
    
    try:
        return legal_research_function(query, jurisdiction)
    except Exception as e:
        print(f"Error in legal research: {e}")
        return f"An error occurred during {jurisdiction} legal research: {str(e)}"

Legal research function created successfully.


# Step 8: Run Main Agent Monitoring Loop & Display

Iterate through the generated itinerary (e.g., daily or by activity).

For each relevant item (flight, outdoor activity, specific attraction), call the search_real_time_info function.

Call analyze_information_for_disruption using the search results and the item.

If a disruption is found, call suggest_and_evaluate_alternatives.

(Crucial Missing Piece): Add logic to update the context_cache["current_itinerary"] dictionary based on the chosen alternative. This requires parsing the suggestion and modifying the JSON structure.

Include time.sleep() to simulate the passage of time and avoid hitting API rate limits.

# Step 9: Enhancements and Considerations

Real Search Integration: Replace search_real_time_info with actual calls to Google Search API (e.g., Custom Search JSON API requiring its own API key and setup) or specialized APIs (WeatherAPI, FlightStats, etc.). Libraries like requests or specific client libraries (google-api-python-client, serpapi) can be used.
Robust Itinerary Updates: Implement the logic to parse the AI's suggested alternatives and reliably update the JSON itinerary structure. This might involve asking the AI to format its suggestions more predictably or using NLP techniques to extract key information.
User Interaction: For confirming alternatives, the simulation currently skips this. A real application would pause and present options to the user.
Error Handling: Add more robust error handling for API calls, JSON parsing, network issues, and unexpected function call arguments.
State Management: For longer-running or more complex agents, saving the context_cache (especially the itinerary) to a file or database between runs might be necessary.
Token Limits: Be mindful of how much context (especially conversation history) you pass back to the model, as exceeding token limits will cause errors. Summarize history if needed.
Cost: Be aware that calls to the Gemini API and potentially external search/data APIs incur costs.
UI: For a user-friendly experience, you'd build a web interface (e.g., using Flask/Django/Streamlit) that interacts with this agent logic running in the background.

In [8]:
# === SECTION 8: DEFINE INTELLIGENT LEGAL ASSISTANT WITH FUNCTION CALLING ===

def legal_assistant_with_functions(query):
    """
    Main legal assistant function that uses function calling when appropriate.
    Now supports both Kenyan and German legal systems.
    """
    global model
    
    print(f"\n--- Processing Legal Query ---")
    print(f"Query: {query}")
    
    # Try to determine jurisdiction from the query
    jurisdiction_keywords = {
        "kenya": ["kenya", "kenyan", "nairobi", "mombasa", "ksh", "east africa", "kenyatta", "odinga", "kikuyu", "swahili", "uhuru", "eacc", "ekrla"],
        "germany": ["germany", "german", "deutschland", "deutsch", "berlin", "munich", "bundesgerichtshof", "bgh", "bverfg", "bundesverfassungsgericht", "bgb", "hamburg", "frankfurt", "bayern", "euro", "eu law", "european union"]
    }
    
    # Default to the user's preferred jurisdiction or Kenya if not set
    user_jurisdiction = context_cache["user_preferences"].get("jurisdiction", "kenya").lower()
    
    # Check if query contains jurisdiction-specific keywords
    query_lower = query.lower()
    detected_jurisdiction = None
    
    for jur, keywords in jurisdiction_keywords.items():
        if any(keyword in query_lower for keyword in keywords):
            detected_jurisdiction = jur
            break
    
    # Use detected jurisdiction if found, otherwise use user preference
    active_jurisdiction = detected_jurisdiction or user_jurisdiction
    print(f"Active jurisdiction: {active_jurisdiction.upper()}")
    
    # Update system prompt with jurisdiction-specific information
    if active_jurisdiction == "germany":
        system_prompt = """
        You are an expert German legal assistant (Rechtsassistent) with access to German legal information tools.
        Your role is to help legal professionals with research, case summaries, and document drafting 
        related to German law (Deutsches Recht).
        Use the available tools when needed to ground your responses in German legal authority.
        Always include sources and citations when providing German legal information.
        Use appropriate German legal terminology with English translations where helpful.
        """
    else:  # Kenya
        system_prompt = """
        You are an expert Kenyan legal assistant with access to legal information tools.
        Your role is to help legal professionals with research, case summaries, and document drafting.
        Use the available tools when needed to ground your responses in Kenyan legal authority.
        Always include sources and citations when providing legal information.
        """
    
    # Store the active jurisdiction in context for this conversation
    context_cache["user_preferences"]["jurisdiction"] = active_jurisdiction
    
    try:
        # Initial call to model with function definitions
        response = model.generate_content(
            [
                {"role": "system", "parts": [system_prompt]},
                {"role": "user", "parts": [query]}
            ],
            tools=tools  # Use the tools defined in section 6
        )
        
        # Process function calls if needed
        function_call_count = 0  # Prevent infinite loops
        max_function_calls = 5
        
        while (function_call_count < max_function_calls):
            # Check if there's a function call in the response
            function_call = None
            try:
                if response.candidates and response.candidates[0].content.parts:
                    for part in response.candidates[0].content.parts:
                        if hasattr(part, 'function_call') and part.function_call:
                            function_call = part.function_call
                            break
            except (AttributeError, IndexError):
                # No function call found
                pass
                
            if not function_call:
                break  # No function call, exit the loop
            
            function_name = function_call.name
            args = {key: value for key, value in function_call.args.items()}
            
            # Add jurisdiction to args if not explicitly provided
            if "jurisdiction" not in args and function_name in ["search_legal_information", "analyze_case_law", "generate_legal_document"]:
                args["jurisdiction"] = active_jurisdiction
                
            print(f"--- Function Call: {function_name}({args}) ---")
            
            if function_name in available_functions:
                function_to_call = available_functions[function_name]
                try:
                    if function_name == "generate_legal_document" and isinstance(args.get("details"), str):
                        # Parse JSON string to dict if needed
                        try:
                            args["details"] = json.loads(args["details"])
                        except:
                            args["details"] = {"issue": args["details"]}
                            
                    function_response_content = function_to_call(**args)
                    
                    # Convert to string for JSON serialization if needed
                    if not isinstance(function_response_content, str):
                        function_response_content = json.dumps(function_response_content)
                        
                except Exception as e:
                    print(f"ERROR: Function execution error: {e}")
                    traceback.print_exc()
                    function_response_content = json.dumps({"error": f"Function execution error: {str(e)}"})
                    
                # Provide function response back to the model
                try:
                    # Format for compatibility with current Google GenAI library
                    response = model.generate_content(
                        [
                            {"role": "system", "parts": [system_prompt]},
                            {"role": "user", "parts": [query]},
                            {
                                "role": "model", 
                                "parts": [{"function_call": function_call}]
                            },
                            {
                                "role": "function",
                                "parts": [{
                                    "function_response": {
                                        "name": function_name,
                                        "response": {"content": function_response_content}
                                    }
                                }]
                            }
                        ],
                        tools=tools
                    )
                except Exception as e:
                    print(f"ERROR: Function response processing failed: {e}")
                    traceback.print_exc()
                    break
            else:
                print(f"ERROR: Unknown function: {function_name}")
                break
                
            function_call_count += 1
        
        # Extract final response
        final_response = response.text if hasattr(response, 'text') else "Error: Could not extract final response."
        
        # Add jurisdiction information to the response if not already included
        jurisdiction_mention = f"based on {active_jurisdiction.title()} law" 
        if jurisdiction_mention.lower() not in final_response.lower():
            final_response += f"\n\n[This response is {jurisdiction_mention}]"
        
        # Add to conversation history
        context_cache["conversation_history"].append({
            "role": "user", 
            "parts": query,
            "jurisdiction": active_jurisdiction
        })
        context_cache["conversation_history"].append({
            "role": "model", 
            "parts": final_response,
            "jurisdiction": active_jurisdiction
        })
        
        return final_response
        
    except Exception as e:
        print(f"ERROR: Assistant processing failed: {e}")
        traceback.print_exc()
        return f"I encountered a technical issue while processing your request for {active_jurisdiction.title()} legal information: {str(e)}"

**Section 9**

In [9]:
# === SECTION 9: CREATE HTML FORMATTING FOR LEGAL RESULTS ===

def format_legal_response_html(response_text, title="Legal Research Results", jurisdiction="kenya"):
    """
    Format a legal response as HTML for better readability.
    Now supports both Kenyan and German legal formatting.
    
    Args:
        response_text: The text to format
        title: The title for the HTML document
        jurisdiction: 'kenya' or 'germany'
    
    Returns:
        Formatted HTML string
    """
    # Basic CSS styling with jurisdiction-specific elements
    base_styles = """
    <style>
        body { font-family: 'Georgia', serif; line-height: 1.6; margin: 0; padding: 20px; color: #333; background-color: #f9f9f9; }
        .legal-document { max-width: 900px; margin: 0 auto; background: white; padding: 30px; box-shadow: 0 2px 10px rgba(0,0,0,0.1); border-radius: 5px; }
        h1 { color: #1a365d; border-bottom: 1px solid #eee; padding-bottom: 10px; margin-bottom: 20px; }
        h2 { color: #2a4365; margin-top: 30px; border-left: 4px solid #3182ce; padding-left: 10px; }
        h3 { color: #2c5282; }
        p { margin-bottom: 15px; text-align: justify; }
        ul, ol { margin-bottom: 20px; }
        li { margin-bottom: 8px; }
        blockquote { border-left: 3px solid #3182ce; padding: 10px 20px; margin: 0 0 20px; background-color: #ebf8ff; }
        code { font-family: monospace; background: #f0f0f0; padding: 2px 4px; border-radius: 3px; }
        table { width: 100%; border-collapse: collapse; margin: 20px 0; }
        th, td { border: 1px solid #e2e8f0; padding: 8px 12px; text-align: left; }
        th { background-color: #edf2f7; }
        .document-section { border: 1px solid #e2e8f0; padding: 15px; margin: 20px 0; border-radius: 4px; }
        
        /* Jurisdiction badge */
        .jurisdiction-badge {
            display: inline-block;
            padding: 3px 8px;
            font-size: 0.8em;
            font-weight: bold;
            color: white;
            border-radius: 3px;
            margin-bottom: 15px;
        }
    </style>
    """
    
    # Kenya-specific styles
    kenya_styles = """
    <style>
        /* Kenya colors */
        .jurisdiction-badge.kenya { background-color: #006600; } /* Green from Kenya flag */
        .case-citation { font-style: italic; color: #2c5282; }
        .statute-reference { font-weight: bold; color: #2b6cb0; }
        .legal-principle { background-color: #ebf8ff; padding: 10px; border-radius: 4px; margin-bottom: 15px; }
        
        /* Kenya-specific header styling */
        .kenya-header { 
            border-top: 5px solid #000000; /* Black from Kenya flag */
            border-bottom: 5px solid #bb0000; /* Red from Kenya flag */
        }
    </style>
    """
    
    # German-specific styles
    german_styles = """
    <style>
        /* German colors */
        .jurisdiction-badge.germany { background-color: #DD0000; } /* Red from German flag */
        .german-border {
            border-top: 5px solid #000000; /* Black from German flag */
            border-bottom: 5px solid #FFCE00; /* Gold from German flag */
        }
        
        /* German legal citation formatting */
        .bgh-citation { font-weight: bold; color: #DD0000; }
        .bverfg-citation { font-weight: bold; color: #8B0000; }
        .paragraph-reference { font-family: monospace; background-color: #FFF9C4; padding: 2px 4px; border-radius: 3px; }
        
        /* German legal terminology formatting */
        .german-term { font-style: italic; }
        .german-section { background-color: #FFF3E0; padding: 10px; border-radius: 4px; margin-bottom: 15px; }
    </style>
    """
    
    # Select jurisdiction-specific styling
    if jurisdiction == "germany":
        jurisdiction_styles = german_styles
        jurisdiction_class = "germany german-border"
        jurisdiction_name = "Germany"
    else:
        jurisdiction_styles = kenya_styles
        jurisdiction_class = "kenya kenya-header"
        jurisdiction_name = "Kenya"
    
    # Process markdown-like elements in the text with jurisdiction awareness
    def process_text(text, jurisdiction):
        # Convert markdown headers
        text = re.sub(r'(?m)^# (.*?)$', r'<h1>\1</h1>', text)
        text = re.sub(r'(?m)^## (.*?)$', r'<h2>\1</h2>', text)
        text = re.sub(r'(?m)^### (.*?)$', r'<h3>\1</h3>', text)
        
        # Process lists
        text = re.sub(r'(?m)^- (.*?)$', r'<li>\1</li>', text)
        text = re.sub(r'(?m)^(\d+)\\. (.*?)$', r'<li>\2</li>', text)
        text = re.sub(r'(<li>.*?</li>\\n)+', r'<ul>\n\g<0></ul>', text)
        
        if jurisdiction == "germany":
            # Format German case citations
            text = re.sub(r'BGH ([IVXLCDM]+) ZR (\d+/\d+)', r'<span class="bgh-citation">BGH \1 ZR \2</span>', text)
            text = re.sub(r'BVerfG (\d+) BvR (\d+/\d+)', r'<span class="bverfg-citation">BVerfG \1 BvR \2</span>', text)
            
            # Format German legal code references
            text = re.sub(r'(§\s*\d+[a-z]?|§§\s*\d+[a-z]?(?:\s*,\s*\d+[a-z]?)*)\s+BGB', 
                          r'<span class="paragraph-reference">\1 BGB</span>', text)
            text = re.sub(r'(Art\.\s*\d+[a-z]?)\s+GG', 
                          r'<span class="paragraph-reference">\1 GG</span>', text)
            
            # Format German terminology
            text = re.sub(r'(Rechtsfrage|Sachverhalt|Entscheidungsgründe|Tenor|Ergebnis|Rechtsgrundsätze)', 
                          r'<span class="german-term">\1</span>', text)
            
            # Format German sections
            text = re.sub(r'(?s)(Rechtliche Würdigung|Rechtliche Prüfung):(.*?)(?=\n\n|$)', 
                          r'<div class="german-section"><strong>\1:</strong>\2</div>', text)
            
        else:  # Kenya
            # Format Kenyan citations and references
            text = re.sub(r'\[(\d{4})\]\s+eKLR', r'<span class="case-citation">[\1] eKLR</span>', text)
            text = re.sub(r'(Article \d+)', r'<span class="statute-reference">\1</span>', text)
            text = re.sub(r'(Section \d+)', r'<span class="statute-reference">\1</span>', text)
            
            # Format Kenyan legal principles
            text = re.sub(r'(?s)Legal Principle:(.*?)(?=\n\n|$)', 
                          r'<div class="legal-principle"><strong>Legal Principle:</strong>\1</div>', text)
        
        # Format paragraphs (common to both)
        text = re.sub(r'(?m)^(?!<[hou]|<li|<div|<p)(.+?)\n\n', r'<p>\1</p>\n\n', text)
        
        return text
    
    # Generate the HTML document
    html_parts = [f"""
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="UTF-8">
        <title>{title} - {jurisdiction_name}</title>
        {base_styles}
        {jurisdiction_styles}
    </head>
    <body>
        <div class="legal-document {jurisdiction_class}">
            <h1>{title}</h1>
            <div class="jurisdiction-badge {jurisdiction}">{jurisdiction_name} Law</div>
            {process_text(response_text, jurisdiction)}
        </div>
    </body>
    </html>
    """]
    
    return "".join(html_parts)

# Section 10

In [10]:
# === SECTION 10: DEMONSTRATION OF LEGAL ASSISTANT CAPABILITIES ===

def demonstrate_legal_assistant():
    """
    Demonstrate the capabilities of our legal assistant with various example queries
    for both Kenyan and German jurisdictions.
    """
    print("\n" + "="*80)
    print("MULTILINGUAL LEGAL ASSISTANT: CAPABILITY DEMONSTRATION")
    print("="*80 + "\n")
    
    # Example 1: Vector store search for both jurisdictions
    print("\n1. VECTOR STORE SEARCH DEMONSTRATION\n")
    
    # Kenya search
    print("=== KENYA SEARCH ===")
    search_query_kenya = "What are the requirements for proving robbery with violence in Kenya?"
    print(f"Query: {search_query_kenya}")
    
    try:
        search_results_kenya = search_legal_information("case_law", search_query_kenya, jurisdiction="kenya")
        print(f"\nKenya Search Results:")
        for i, result in enumerate(search_results_kenya["results"]):
            print(f"  Result {i+1}: {result['title']}")
            print(f"  Snippet: {result['snippet'][:150]}...\n")
    except Exception as e:
        print(f"Error during Kenya vector store search: {e}")
    
    # Germany search
    print("\n=== GERMANY SEARCH ===")
    search_query_germany = "What are the requirements for security deposits in German rental law?"
    print(f"Query: {search_query_germany}")
    
    try:
        search_results_germany = search_legal_information("case_law", search_query_germany, jurisdiction="germany")
        print(f"\nGermany Search Results:")
        for i, result in enumerate(search_results_germany["results"]):
            print(f"  Result {i+1}: {result['title']}")
            print(f"  Snippet: {result['snippet'][:150]}...\n")
    except Exception as e:
        print(f"Error during Germany vector store search: {e}")
    
    # Example 2: Legal assistant with function calling for both jurisdictions
    print("\n2. LEGAL ASSISTANT WITH FUNCTION CALLING\n")
    
    # Kenya query
    print("=== KENYA QUERY ===")
    legal_query_kenya = "What are the constitutional protections for freedom of assembly in Kenya?"
    print(f"Query: {legal_query_kenya}")
    
    try:
        # Set jurisdiction in context cache
        context_cache["user_preferences"]["jurisdiction"] = "kenya"
        response_kenya = legal_assistant_with_functions(legal_query_kenya)
        print(f"\nLegal Assistant Response (Kenya):")
        print(response_kenya[:500] + "...\n[Response truncated for brevity]" if len(response_kenya) > 500 else response_kenya)
    except Exception as e:
        print(f"Error with Kenyan function-calling assistant: {e}")
    
    # Germany query
    print("\n=== GERMANY QUERY ===")
    legal_query_germany = "What are the constitutional protections for freedom of expression in Germany?"
    print(f"Query: {legal_query_germany}")
    
    try:
        # Set jurisdiction in context cache
        context_cache["user_preferences"]["jurisdiction"] = "germany"
        response_germany = legal_assistant_with_functions(legal_query_germany)
        print(f"\nLegal Assistant Response (Germany):")
        print(response_germany[:500] + "...\n[Response truncated for brevity]" if len(response_germany) > 500 else response_germany)
    except Exception as e:
        print(f"Error with German function-calling assistant: {e}")
    
    # Example 3: Legal document generation for both jurisdictions
    print("\n3. LEGAL DOCUMENT GENERATION\n")
    
    # Kenya document
    print("=== KENYA DOCUMENT ===")
    kenya_document_details = {
        "client_name": "Mary Otieno",
        "opposing_party": "Nairobi County Government",
        "issue": "Ownership dispute over land claimed by county as public space",
        "client_claim": "Purchased land from original allottee in 1995 with valid title deed",
        "relief_sought": "Declaration of ownership and permanent injunction against interference"
    }
    print(f"Generating Kenyan legal memo for land ownership dispute...")
    
    try:
        kenya_document = generate_legal_document("legal_memo", kenya_document_details, jurisdiction="kenya")
        print(f"\nGenerated Kenyan Legal Memo:")
        print(kenya_document[:500] + "...\n[Document truncated for brevity]")
        
        # Instead of using display(HTML()), which might trigger database writes
        # just print a message indicating the HTML would be displayed
        print("\nHTML Formatted Kenyan Memo would be displayed here in notebook...")
    except Exception as e:
        print(f"Error generating Kenyan legal document: {e}")
    
    # Germany document
    print("\n=== GERMANY DOCUMENT ===")
    german_document_details = {
        "client_name": "Thomas Müller",
        "opposing_party": "Vermieterin GmbH",
        "issue": "Dispute over rental deposit retention after lease termination",
        "client_claim": "Landlord withheld security deposit without documenting damages",
        "relief_sought": "Return of full security deposit plus interest"
    }
    print(f"Generating German legal opinion (Rechtsgutachten) for rental deposit dispute...")
    
    try:
        german_document = generate_legal_document("rechtsgutachten", german_document_details, jurisdiction="germany")
        print(f"\nGenerated German Legal Opinion:")
        print(german_document[:500] + "...\n[Document truncated for brevity]")
        
        print("\nHTML Formatted German Opinion would be displayed here in notebook...")
    except Exception as e:
        print(f"Error generating German legal document: {e}")
    
    # Example 4: Comprehensive Legal Research for both jurisdictions
    print("\n4. COMPREHENSIVE LEGAL RESEARCH\n")
    
    # Kenya research
    print("=== KENYA RESEARCH ===")
    complex_query_kenya = "What legal standards are used by Kenyan courts to determine land ownership in disputes between private individuals and county governments?"
    print(f"Query: {complex_query_kenya}")
    
    # Germany research
    print("\n=== GERMANY RESEARCH ===")
    complex_query_germany = "What legal standards are used by German courts to determine landlord liability for tenant property damage claims?"
    print(f"Query: {complex_query_germany}")
    
    # Check if legal research function exists and run it for both jurisdictions
    if 'legal_research_function' in globals() and legal_research_function:
        try:
            print("Running legal research function for Kenya...")
            research_response_kenya = run_legal_research(complex_query_kenya, jurisdiction="kenya")
            print("\nKenya Legal Research Response (First 500 characters):")
            print(research_response_kenya[:500] + "...\n[Response truncated for brevity]")
            
            print("\nRunning legal research function for Germany...")
            research_response_germany = run_legal_research(complex_query_germany, jurisdiction="germany")
            print("\nGermany Legal Research Response (First 500 characters):")
            print(research_response_germany[:500] + "...\n[Response truncated for brevity]")
            
            # Avoid HTML display which might trigger database writes
            print("\nHTML Formatted Research would be displayed here in notebook...")
        except Exception as e:
            print(f"Error demonstrating legal research: {e}")
            print("The research demonstration has been skipped.")
    else:
        print("Legal research function is not available. Skipping demonstration.")
    
    # Example 5: Comparative Legal Analysis
    print("\n5. COMPARATIVE LEGAL ANALYSIS\n")
    comparative_query = "Compare the constitutional protections for freedom of expression in Kenya and Germany."
    print(f"Query: {comparative_query}")
    
    try:
        # This would typically call a specialized comparative analysis function, but for demonstration
        # we'll use the standard legal assistant with function calling
        print("This would call a comparative analysis function in a full implementation.")
        print("For demonstration purposes, we'll skip executing this to avoid lengthy processing.")
    except Exception as e:
        print(f"Error with comparative analysis: {e}")
    
    print("\n" + "="*80)
    print("DEMONSTRATION COMPLETED")
    print("="*80 + "\n")

# Explicitly print a message instead of running the function
print("Multi-jurisdictional demonstration function defined. To run, use demonstrate_legal_assistant()")
print("Note: Some display features are disabled to avoid database write errors in Kaggle.")

Multi-jurisdictional demonstration function defined. To run, use demonstrate_legal_assistant()
Note: Some display features are disabled to avoid database write errors in Kaggle.


# Section 11

In [11]:
# === SECTION 11: USER INTERFACE WITH GRADIO ===

!pip install -q gradio

import gradio as gr

def create_legal_assistant_ui():
    """
    Create a simple interactive UI for the multi-jurisdictional Legal Assistant using Gradio.
    Supports both Kenyan and German legal research and document generation.
    """
    # Define the legal research function with jurisdiction support
    def research_function(query, jurisdiction):
        if not query:
            return "Please enter a legal question."
        
        try:
            # Update user preferences with selected jurisdiction
            context_cache["user_preferences"]["jurisdiction"] = jurisdiction
            
            # Determine whether to use the specialized legal research function or the general assistant
            if 'legal_research_function' in globals() and legal_research_function and query.lower().startswith("research:"):
                clean_query = query[9:].strip()  # Remove the "research:" prefix
                result = run_legal_research(clean_query, jurisdiction)
            else:
                result = legal_assistant_with_functions(query)
                
            return result
        except Exception as e:
            error_msg = str(e)
            print(f"Error in research function: {error_msg}")
            return f"Error processing your request: {error_msg}"
    
    # Define the case analysis function with jurisdiction support
    def analyze_function(case_text, analysis_type, jurisdiction):
        if not case_text:
            return "Please enter case text to analyze."
        
        try:
            analysis = analyze_case_law(case_text, analysis_type.lower(), jurisdiction)
            return analysis
        except Exception as e:
            print(f"Error in analyze function: {str(e)}")
            return f"Error analyzing case: {str(e)}"
    
    # Define the document generation function with jurisdiction support
    def generate_document_function(
        document_type, jurisdiction, client_name, opposing_party, case_issue, 
        client_claim, relief_sought
    ):
        if not client_name or not case_issue:
            return "Please enter at least the client name and case issue."
        
        try:
            details = {
                "client_name": client_name,
                "opposing_party": opposing_party,
                "issue": case_issue,
                "client_claim": client_claim,
                "relief_sought": relief_sought
            }
            
            # Map document types from UI to backend function
            doc_type_mapping = {
                "Legal Memo (Kenya)": "legal_memo",
                "Petition (Kenya)": "petition",
                "Affidavit (Kenya)": "affidavit",
                "Legal Opinion (Germany)": "rechtsgutachten",
                "Lawsuit Filing (Germany)": "klage",
                "Statutory Declaration (Germany)": "eidesstattliche_versicherung"
            }
            
            backend_doc_type = doc_type_mapping.get(document_type, "legal_memo")
            document = generate_legal_document(backend_doc_type, details, jurisdiction)
            return document
        except Exception as e:
            print(f"Error in document generation: {str(e)}")
            return f"Error generating document: {str(e)}"
    
    # Create the interface
    with gr.Blocks(title="Multilingual Legal Assistant") as interface:
        gr.Markdown(
            """
            # Multilingual Legal Assistant
            ## Intelligent Case Summaries & Drafting Tool for Kenya & Germany
            
            A capstone project for the 5-Day Google Generative AI Intensive Course
            """
        )
        
        with gr.Tab("Legal Research"):
            with gr.Row():
                with gr.Column():
                    jurisdiction_selector = gr.Radio(
                        label="Jurisdiction",
                        choices=["kenya", "germany"],
                        value="kenya"
                    )
                    
                    query_input = gr.Textbox(
                        label="Legal Question", 
                        placeholder="Enter your legal question here... (Start with 'research:' for in-depth research)",
                        lines=3
                    )
                    
                    research_type = gr.Checkbox(
                        label="Use comprehensive research mode", 
                        value=False,
                        info="This will use the specialized legal research function for more in-depth analysis"
                    )
                    
                    research_button = gr.Button("Research")
                
                with gr.Column():
                    research_output = gr.Markdown(label="Legal Analysis")
            
            # Update the query when research mode is toggled
            def update_research_prefix(text, use_research):
                if use_research and not text.lower().startswith("research:"):
                    return f"research: {text}"
                elif not use_research and text.lower().startswith("research:"):
                    return text[9:].strip()
                return text
            
            research_type.change(
                fn=update_research_prefix,
                inputs=[query_input, research_type],
                outputs=query_input
            )
            
            research_button.click(
                fn=research_function,
                inputs=[query_input, jurisdiction_selector],
                outputs=research_output
            )
        
        with gr.Tab("Case Analysis"):
            with gr.Row():
                with gr.Column():
                    case_jurisdiction = gr.Radio(
                        label="Case Jurisdiction",
                        choices=["kenya", "germany"],
                        value="kenya"
                    )
                    
                    case_input = gr.Textbox(
                        label="Case Text", 
                        placeholder="Paste the case text here...",
                        lines=10
                    )
                    
                    analysis_type = gr.Radio(
                        label="Analysis Type",
                        choices=["Brief", "Comprehensive", "Legal Principles"],
                        value="Comprehensive"
                    )
                    
                    analyze_button = gr.Button("Analyze Case")
                
                with gr.Column():
                    analysis_output = gr.Markdown(label="Case Analysis")
            
            # Set jurisdiction-specific example text
            def set_example_case(jurisdiction):
                if jurisdiction == "kenya":
                    return """REPUBLIC OF KENYA
IN THE HIGH COURT OF KENYA AT NAIROBI
CONSTITUTIONAL AND HUMAN RIGHTS DIVISION
PETITION NO. 123 OF 2022

BETWEEN

JOHN SMITH....................................................................PETITIONER

AND

THE REPUBLIC OF KENYA.....................................................RESPONDENT

JUDGMENT

Introduction:
1. The Petitioner, John Smith, filed this petition on 15th March 2022 challenging the constitutionality of Section 24 of the Public Order Act.

Background:
2. The Petitioner was arrested during a peaceful demonstration at Uhuru Park, Nairobi, on 10th February 2022.
3. The Petitioner claims that the arrest violated his constitutional rights to freedom of assembly and expression.

[Case excerpt for analysis]"""
                else:
                    return """BUNDESGERICHTSHOF
IM NAMEN DES VOLKES
URTEIL
VIII ZR 42/20

In dem Rechtsstreit

[Anonymisiert] - Kläger und Revisionskläger

gegen

[Anonymisiert] - Beklagte und Revisionsbeklagte

hat der VIII. Zivilsenat des Bundesgerichtshofs auf die mündliche Verhandlung
vom 12. Mai 2021 durch [Richtername] als Vorsitzenden, die Richter [Namen]
und die Richterinnen [Namen] für Recht erkannt:

Tenor:
Auf die Revision des Klägers wird das Urteil des Landgerichts München I
vom 21. Januar 2020 aufgehoben. Die Sache wird zur neuen Verhandlung
und Entscheidung an das Berufungsgericht zurückverwiesen.

[Case excerpt for analysis]"""
            
            case_jurisdiction.change(
                fn=set_example_case,
                inputs=case_jurisdiction,
                outputs=case_input
            )
            
            analyze_button.click(
                fn=analyze_function,
                inputs=[case_input, analysis_type, case_jurisdiction],
                outputs=analysis_output
            )
        
        with gr.Tab("Document Generation"):
            with gr.Row():
                with gr.Column():
                    doc_jurisdiction = gr.Radio(
                        label="Document Jurisdiction",
                        choices=["kenya", "germany"],
                        value="kenya"
                    )
                    
                    doc_type = gr.Dropdown(
                        label="Document Type",
                        choices=[
                            "Legal Memo (Kenya)", 
                            "Petition (Kenya)", 
                            "Affidavit (Kenya)",
                            "Legal Opinion (Germany)",
                            "Lawsuit Filing (Germany)",
                            "Statutory Declaration (Germany)"
                        ],
                        value="Legal Memo (Kenya)"
                    )
                    
                    client_name = gr.Textbox(
                        label="Client Name", 
                        value="John Kamau"
                    )
                    
                    opposing_party = gr.Textbox(
                        label="Opposing Party", 
                        value="Nairobi County Government"
                    )
                    
                with gr.Column():
                    case_issue = gr.Textbox(
                        label="Case Issue", 
                        value="Land ownership dispute over property in Westlands"
                    )
                    
                    client_claim = gr.Textbox(
                        label="Client's Claim", 
                        value="Client has owned the property since 1995 with valid title deed"
                    )
                    
                    relief_sought = gr.Textbox(
                        label="Relief Sought", 
                        value="Declaration of ownership and permanent injunction against interference"
                    )
            
            generate_button = gr.Button("Generate Document")
            document_output = gr.Textbox(
                label="Generated Document", 
                lines=20
            )
            
            # Update document type choices based on jurisdiction selection
            def update_doc_types(jurisdiction):
                if jurisdiction == "kenya":
                    return gr.Dropdown.update(
                        choices=[
                            "Legal Memo (Kenya)", 
                            "Petition (Kenya)", 
                            "Affidavit (Kenya)"
                        ],
                        value="Legal Memo (Kenya)"
                    )
                else:
                    return gr.Dropdown.update(
                        choices=[
                            "Legal Opinion (Germany)",
                            "Lawsuit Filing (Germany)",
                            "Statutory Declaration (Germany)"
                        ],
                        value="Legal Opinion (Germany)"
                    )
            
            # Update example values based on jurisdiction selection
            def update_examples(jurisdiction):
                if jurisdiction == "kenya":
                    return [
                        gr.Textbox.update(value="John Kamau"),
                        gr.Textbox.update(value="Nairobi County Government"),
                        gr.Textbox.update(value="Land ownership dispute over property in Westlands"),
                        gr.Textbox.update(value="Client has owned the property since 1995 with valid title deed"),
                        gr.Textbox.update(value="Declaration of ownership and permanent injunction against interference")
                    ]
                else:
                    return [
                        gr.Textbox.update(value="Thomas Müller"),
                        gr.Textbox.update(value="Vermieterin GmbH"),
                        gr.Textbox.update(value="Dispute over rental deposit retention after lease termination"),
                        gr.Textbox.update(value="Landlord withheld security deposit without documenting damages"),
                        gr.Textbox.update(value="Return of full security deposit plus interest")
                    ]
            
            # Connect jurisdiction radio to dynamic UI updates
            doc_jurisdiction.change(
                fn=update_doc_types,
                inputs=doc_jurisdiction,
                outputs=doc_type
            )
            
            doc_jurisdiction.change(
                fn=update_examples,
                inputs=doc_jurisdiction,
                outputs=[client_name, opposing_party, case_issue, client_claim, relief_sought]
            )
            
            generate_button.click(
                fn=generate_document_function,
                inputs=[
                    doc_type, doc_jurisdiction, client_name, opposing_party, case_issue,
                    client_claim, relief_sought
                ],
                outputs=document_output
            )
        
        with gr.Tab("Comparative Analysis"):
            with gr.Row():
                with gr.Column():
                    comparative_query = gr.Textbox(
                        label="Comparative Legal Question", 
                        placeholder="Enter a question that compares legal aspects between Kenya and Germany...",
                        lines=3,
                        value="Compare constitutional protections for freedom of expression in Kenya and Germany."
                    )
                    
                    comparative_button = gr.Button("Analyze")
                
                with gr.Column():
                    comparative_output = gr.Markdown(label="Comparative Analysis")
            
            # Define the comparative analysis function
            def comparative_analysis(query):
                if not query:
                    return "Please enter a comparative legal question."
                
                try:
                    # This would typically be handled by a specialized function
                    # For now, we'll use the general assistant with appropriate prompt engineering
                    enhanced_query = f"Perform a detailed comparative legal analysis: {query}. Compare the approaches in both Kenyan and German legal systems, highlighting similarities, differences, and underlying legal principles."
                    
                    # Reset jurisdiction to avoid bias
                    context_cache["user_preferences"]["jurisdiction"] = "both"
                    
                    result = legal_assistant_with_functions(enhanced_query)
                    return result
                except Exception as e:
                    print(f"Error in comparative analysis: {str(e)}")
                    return f"Error processing comparative analysis: {str(e)}"
            
            comparative_button.click(
                fn=comparative_analysis,
                inputs=comparative_query,
                outputs=comparative_output
            )
        
        with gr.Tab("About"):
            gr.Markdown(
                """
                ## About Multilingual Legal Assistant
                
                This tool was developed as a capstone project for the 5-Day Google Generative AI Intensive Course.
                
                ### Features
                - **Multilingual Legal Research**: Search and analyze both Kenyan and German case law using vector search and Google Search
                - **Case Summarization**: Generate structured summaries of legal cases in both jurisdictions
                - **Document Generation**: Create professional legal documents following jurisdiction-specific formats and conventions
                - **Comparative Analysis**: Research legal topics across different legal systems
                
                ### Technologies Used
                - Google's Gemini Pro for natural language generation
                - Vector embeddings for semantic search
                - LangChain for building the legal assistant
                - LangGraph for agent-based legal research
                - RAG (Retrieval-Augmented Generation) for grounded legal analysis
                
                ### Data Sources
                The system uses sample legal documents from both jurisdictions:
                - Kenyan case law focused on constitutional, criminal, and land law cases
                - German case law focused on civil, constitutional, and contract law cases
                """
            )
    
    return interface

# Create and launch the interface (commented out for Kaggle notebook submission)
demo = create_legal_assistant_ui()
demo.launch()

print("Multi-jurisdictional UI code generated successfully!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 32.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 93.7 MB/s eta 0:00:00:00:010:01
* Running on local URL:  http://127.0.0.1:7860
Kaggle notebooks require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://c7f1c44ec076a638be.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Multi-jurisdictional UI code generated successfully!


# Section 12: Project Summary

In [12]:
# === SECTION 12: KAGGLE CAPSTONE PROJECT SUMMARY ===

"""
MULTILINGUAL LEGAL ASSISTANT: KAGGLE CAPSTONE PROJECT SUMMARY

This notebook demonstrates a comprehensive application of the concepts learned in the 
5-Day Google Generative AI Intensive Course through the creation of a specialized legal 
assistant that supports both Kenyan and German legal systems.

KEY ACHIEVEMENTS:

1. Implemented a multi-jurisdictional vector store for semantic search of both Kenyan and German
   legal documents, enabling accurate retrieval of relevant case law and statutes across legal systems.

2. Integrated Google Search API grounding to supplement the vector stores with up-to-date
   legal information from authoritative sources in both jurisdictions.

3. Created a LangGraph agent that orchestrates a multi-step workflow for complex legal
   research tasks across different legal systems, demonstrating agent-based reasoning.

4. Implemented function calling with Gemini Pro to enable the model to perform specific
   legal tasks with jurisdiction awareness, including document generation and case analysis.

5. Built a retrieval-augmented generation (RAG) system that ensures all legal advice is
   grounded in authoritative sources from the appropriate legal system.

6. Developed specialized legal document generation capabilities with proper formatting
   and citation styles for both Kenyan and German legal documents.

7. Created a user-friendly Gradio interface that dynamically adapts to the selected
   jurisdiction, providing appropriate document types, examples, and guidance.

8. Implemented jurisdiction detection from user queries to automatically route
   requests to the appropriate legal knowledge base.

9. Added comparative legal analysis capabilities to examine legal topics across
   different legal systems, highlighting key similarities and differences.

This project showcases how generative AI can be applied to specialized professional
domains like legal research and document preparation across multiple jurisdictions,
significantly improving efficiency and accessibility of legal services.

REAL-WORLD APPLICATIONS:

1. Legal Aid Organizations: Assisting paralegals and legal aid workers in multiple
   countries with limited access to legal resources and research tools.

2. Law Firms: Accelerating research and document preparation for international legal
   professionals working across different jurisdictions.

3. Judicial System: Assisting judges and court staff in processing, summarizing, and
   researching cases from different legal systems to improve judicial efficiency.

4. Legal Education: Providing accessible resources for law students and researchers
   to understand and compare different legal systems and principles.

5. Access to Justice: Making legal information more accessible to the general public
   across multiple jurisdictions, helping bridge the justice gap.

6. International Organizations: Supporting legal work that spans multiple legal
   systems, such as international trade, human rights, or immigration cases.

7. Comparative Legal Research: Facilitating research across legal systems for
   academic purposes, legal reform initiatives, or policy development.

LESSONS LEARNED:

1. Domain-specific vector stores organized by jurisdiction provide far more accurate
   legal information than a single, undifferentiated knowledge base.

2. LangGraph enables sophisticated, multi-step reasoning across different legal systems
   that would be difficult with a single prompt.

3. Function calling with Gemini Pro allows for specialized legal tasks with jurisdiction
   awareness, enabling appropriate formatting and style conventions.

4. Retrieval-augmented generation is essential for legal applications where accuracy
   and citation to authority are paramount, especially across different legal systems.

5. Legal document formatting and citation styles vary significantly between jurisdictions
   and require careful prompt engineering to match professional standards.

6. User interfaces for legal tools need to adapt dynamically to jurisdiction selection
   to provide relevant options and examples.

7. Jurisdiction detection from user queries can enhance user experience by automatically
   routing to the appropriate knowledge base.

NEXT STEPS:

1. Integration with official legal databases in both Kenya and Germany, such as Kenya Law
   Reports and the German Federal Law Gazette (Bundesgesetzblatt).

2. Expansion to include more jurisdictions, such as UK, US, or EU legal systems.

3. Implementation of authentication and privacy features for handling sensitive
   client information across international boundaries.

4. Development of more sophisticated reasoning capabilities for complex legal analysis
   that spans multiple jurisdictions.

5. Addition of document comparison and revision tracking for legal drafting with
   support for multiple languages.

6. Enhancement of the comparative analysis features with visualization tools to
   highlight key differences between legal systems.

7. Integration with translation capabilities to facilitate cross-lingual legal research.

This capstone project demonstrates the power of combining vector embeddings, LangGraph agents,
Google Search grounding, and Gemini AI to create practical, specialized tools for professionals
working across different legal systems and jurisdictions.
"""

print("Multilingual legal assistant capstone project complete!")

Multilingual legal assistant capstone project complete!


# Section 13: Sample

In [13]:
# === SECTION 13: EXAMPLE USAGE ===

# Example 1: Direct use of Gemini model for legal queries in both jurisdictions
example_query_kenya = "Explain the constitutional provisions for freedom of assembly in Kenya and key court cases interpreting these rights."
example_query_germany = "Explain Article 5 of the German Basic Law (Grundgesetz) regarding freedom of expression and key Federal Constitutional Court cases interpreting these rights."

print("\n=== EXAMPLE 1: DIRECT LEGAL QUERIES ===\n")

print("=== KENYA QUERY ===")
print(f"Query: {example_query_kenya}")

try:
    # Set jurisdiction in context
    context_cache["user_preferences"]["jurisdiction"] = "kenya"
    response_kenya = model.generate_content(example_query_kenya)
    print("\nResponse (Kenya):")
    if hasattr(response_kenya, 'text'):
        print(response_kenya.text[:500] + "..." if len(response_kenya.text) > 500 else response_kenya.text)
    else:
        print("Could not generate response for Kenyan query.")
except Exception as e:
    print(f"Error: {e}")
    print("Skipping this example due to API error.")

print("\n=== GERMANY QUERY ===")
print(f"Query: {example_query_germany}")

try:
    # Set jurisdiction in context
    context_cache["user_preferences"]["jurisdiction"] = "germany"
    response_germany = model.generate_content(example_query_germany)
    print("\nResponse (Germany):")
    if hasattr(response_germany, 'text'):
        print(response_germany.text[:500] + "..." if len(response_germany.text) > 500 else response_germany.text)
    else:
        print("Could not generate response for German query.")
except Exception as e:
    print(f"Error: {e}")
    print("Skipping this example due to API error.")

# Example 2: Using the Legal Research Function for both jurisdictions
print("\n=== EXAMPLE 2: COMPREHENSIVE LEGAL RESEARCH ===\n")

print("=== KENYA RESEARCH ===")
example_research_kenya = "What legal standards are used by Kenyan courts to determine land ownership in disputes between private individuals and government entities?"
print(f"Query: {example_research_kenya}")
print("\nNote: Comprehensive research execution available through run_legal_research(query, jurisdiction='kenya')")

print("\n=== GERMANY RESEARCH ===")
example_research_germany = "What legal standards does the German Federal Court of Justice (BGH) apply in rental deposit disputes?"
print(f"Query: {example_research_germany}")
print("\nNote: Comprehensive research execution available through run_legal_research(query, jurisdiction='germany')")

# Example 3: Generate legal documents for both jurisdictions
print("\n=== EXAMPLE 3: LEGAL DOCUMENT GENERATION ===\n")

print("=== KENYA DOCUMENT ===")
kenya_doc_query = "Draft a petition for a client named James Mwangi who is challenging the constitutionality of his arrest during a peaceful protest."
print(f"Query: {kenya_doc_query}")

try:
    # Use the function directly instead of going through the assistant
    kenya_document_details = {
        "client_name": "James Mwangi",
        "issue": "Challenging constitutionality of arrest during peaceful protest",
        "facts": "Client was arrested while participating in a peaceful environmental protest at Uhuru Park on March 15, 2025",
        "legal_basis": "The arrest violates Articles 33 and 37 of the Constitution of Kenya",
        "relief_sought": "Declaration that the arrest was unconstitutional and damages of KSh. 500,000"
    }
    
    print("Generating Kenyan petition document...")
    kenya_response = generate_legal_document("petition", kenya_document_details, jurisdiction="kenya")
    
    print("\nSample Kenyan Petition Document:")
    if len(kenya_response) > 1000:
        print(kenya_response[:1000] + "...\n[Document truncated for brevity]")
    else:
        print(kenya_response)
except Exception as e:
    print(f"Error generating Kenyan document: {e}")
    print("Skipping document generation due to API error.")

print("\n=== GERMANY DOCUMENT ===")
german_doc_query = "Draft a German legal opinion (Rechtsgutachten) for a client named Thomas Schmidt regarding the legality of his landlord withholding his rental security deposit."
print(f"Query: {german_doc_query}")

try:
    # Use the function directly instead of going through the assistant
    german_document_details = {
        "client_name": "Thomas Schmidt",
        "opposing_party": "Vermietungsgesellschaft Wagner GmbH",
        "issue": "Legality of landlord withholding rental security deposit after lease termination",
        "facts": "Client moved out of apartment on February 28, 2025. Landlord claims unspecified damages but has provided no evidence after 3 months",
        "legal_basis": "Sections 551, 556 of the German Civil Code (BGB)",
        "relief_sought": "Full return of the 3-month security deposit plus interest"
    }
    
    print("Generating German legal opinion document...")
    german_response = generate_legal_document("rechtsgutachten", german_document_details, jurisdiction="germany")
    
    print("\nSample German Legal Opinion Document:")
    if len(german_response) > 1000:
        print(german_response[:1000] + "...\n[Document truncated for brevity]")
    else:
        print(german_response)
except Exception as e:
    print(f"Error generating German document: {e}")
    print("Skipping document generation due to API error.")

# Example 4: Case Analysis for both jurisdictions
print("\n=== EXAMPLE 4: CASE ANALYSIS ===\n")

print("=== KENYA CASE ANALYSIS ===")
print("Sample usage: analyze_case_law(kenyan_case_text, 'comprehensive', 'kenya')")
print("This would provide a structured analysis of a Kenyan legal case.")

print("\n=== GERMANY CASE ANALYSIS ===")
print("Sample usage: analyze_case_law(german_case_text, 'comprehensive', 'germany')")
print("This would provide a structured analysis of a German legal case with appropriate German legal terminology.")

# Example 5: Comparative Legal Analysis
print("\n=== EXAMPLE 5: COMPARATIVE LEGAL ANALYSIS ===\n")
comparative_query = "Compare how freedom of expression is protected and limited in Kenyan and German constitutional law."
print(f"Query: {comparative_query}")

# Fix the syntax error by ensuring proper string closing
print("""
Example implementation:
```python
def perform_comparative_analysis(query):
    enhanced_query = f"Comparative legal analysis: {query}. Compare Kenyan and German approaches."
    context_cache["user_preferences"]["jurisdiction"] = "both"
    return legal_assistant_with_functions(enhanced_query)

SyntaxError: incomplete input (<ipython-input-13-a78d1f5acb50>, line 127)